Rodzaje ruchu:\
Swap - losujemy dwa miasta i zamieniamy je ze sobą.\
Insercja - losujemy miasto i punkt w którym je umieszczamy.\
Zamiana kolejności - losujemy dwa punkty i między nimi zamieniamy kolejność.\
\
Sąsiedztwo to dwa przypadki które różnią się między sobą jednym ruchem\
\
Klasyczny algorytm wspinaczki z multistartem: po każdym ruchu liczymy drogę i sprawdzamy czy jest mniejsza czy nie, wybieramy tylko lepsze opcje. Multistart polega na wielokrotnym odpaleniu algorytmu z losowaniem punktu startowego.\
\
Algorytm symulowanego wyżarzania: zmodyfikowany algorytm wspinaczki, jeśli znalezione rozwiązanie jest gorsze to nadal je rozważamy, na podstawie tego jak bardzo jest gorsze oraz od temperatury.\
Temperatura jest wysoka na początku działania algorytmu i spada razem z czasem wykonywania algorytmu, im niższa temperatura tym mniejsze prawdopodobieństwo wybrania gorszego rozwiązania, bo algorytm już trochę działa i zakładamy że jest w lepszym miejscu niż na początku.\
Pozwala on nam wyjść z niektórych ekstremów lokalnych dzięki możliwości przyjęcia gorszego rozwiązania.\
\
Tabu search:\
Sąsiedztwo wygląda tutaj inaczej: przy swapie losujemy jedną liczbę i zamieniamy wszystkie kombinacje z tą liczbą, ze wszystkich kombinacji wybieramy najlepszą.\
Następnie taka kombinacja trafia na listę tabu, czyli takiej samej zamiany nie można wykonać przez określoną kolejną liczbę ruchów.\
\
Funkcja aspiracji to funkcja, która pod pewnymi warunkami pozwala wykonać ruch z listy tabu ale nie trzeba tego implementować.\
\
Odpowiednia długość listy tabu pozwala wyjść z niektórych ekstremów lokalnych, jeśli będzie za krótka to możemy nie wyjść z ekstremum lokalnego, jeśli będzie za długa to możemy wyjść z ekstremum niepotrzebnie.


In [326]:
import numpy as np
import math

In [187]:
#wspinaczka z multistartem
distance = np.array([[0,2,4,1,5],
                    [2,0,3,8,7],
                    [4,3,0,5,1],
                    [1,8,5,0,3],
                    [5,7,1,3,0]])

In [180]:
def Odleglosc(distance, trasa):
    suma = 0
    for i in range(len(trasa)):
        if i+1 < len(trasa):
            suma += distance[trasa[i]][trasa[i+1]]
        else:
            suma += distance[trasa[0]][trasa[i]]
    return suma

In [335]:
#swap
def Swap(trasa):
    trasa_temp = trasa.copy()
    #print(trasa)
    swap = np.random.choice(trasa_temp,2,replace=False)
    temp = trasa_temp[swap[0]]
    trasa_temp[swap[0]] = trasa_temp[swap[1]]
    trasa_temp[swap[1]] = temp
    #print(swap)
    #print(trasa)
    return trasa_temp

#insercja
def Insercja(trasa):
    trasa_temp = trasa.copy()
    insercja = np.random.choice(trasa_temp,2,replace=False)
    #print(trasa_temp)
    #print(insercja)
    temp = trasa_temp[insercja[0]]
    trasa_temp = np.delete(trasa_temp,insercja[0])
    trasa_temp = np.insert(trasa_temp,insercja[1],temp)
    #print(trasa_temp)
    return trasa_temp

def Zamiana(trasa):
    trasa_temp = trasa.copy()
    zamiana = np.random.choice(trasa_temp,2,replace=False)
    zamiana.sort()
    #print(trasa_temp)
    #print(zamiana)
    for i in range(math.ceil((zamiana[1]-zamiana[0])/2)):
        temp = trasa_temp[zamiana[1]]
        trasa_temp[zamiana[1]] = trasa_temp[zamiana[0]]
        trasa_temp[zamiana[0]] = temp
        zamiana[1] -= 1
        zamiana[0] += 1
    #print(trasa_temp)
    return trasa_temp

In [339]:
#Zamiana(trasa)

In [338]:
#trasa = np.arange(len(distance))
#np.random.shuffle(trasa)
#Insercja(trasa)

In [268]:
#temp = trasa[0] #temp = 4
#trasa = np.delete(trasa,0)
#trasa = np.insert(trasa,4,3)

In [340]:
#ruch polegający na swapie, sprawdzenie sąsiadów, zwrot mniejszej trasy
def Ruch_swap(distance,trasa):
    odl = Odleglosc(distance,trasa)
    trasa_new = Swap(trasa)
    odl_new = Odleglosc(distance,trasa_new)
    if odl_new < odl:
        trasa = trasa_new
    else:
        print("nowa trasa jest dluzsza lub rowna")
    return trasa

#ruch polegający na insercji, sprawdzenie sąsiadów, zwrot mniejszej trasy
def Ruch_insercja(distance,trasa):
    odl = Odleglosc(distance,trasa)
    trasa_new = Insercja(trasa)
    odl_new = Odleglosc(distance,trasa_new)
    if odl_new < odl:
        trasa = trasa_new
    else:
        print("nowa trasa jest dluzsza lub rowna")
    return trasa

#ruch polegający na zamianie, sprawdzenie sąsiadów, zwrot mniejszej trasy
def Ruch_zamiana(distance,trasa):
    odl = Odleglosc(distance,trasa)
    trasa_new = Zamiana(trasa)
    odl_new = Odleglosc(distance,trasa_new)
    if odl_new < odl:
        trasa = trasa_new
    else:
        print("nowa trasa jest dluzsza lub rowna")
    return trasa

In [285]:
#algorytm ze swapem
trasa = np.arange(len(distance))
np.random.shuffle(trasa)
licza_iteracji = 5

print("Start")
print(trasa)
print(Odleglosc(distance,trasa))
for i in range(licza_iteracji):
    print("\n{} iteracja".format(i+1))
    trasa = Ruch_swap(distance,trasa)
    print(trasa)
    print(Odleglosc(distance,trasa))


Start
[3 0 4 2 1]
18

1 iteracja
nowa trasa jest dluzsza lub rowna
[3 0 4 2 1]
18

2 iteracja
nowa trasa jest dluzsza lub rowna
[3 0 4 2 1]
18

3 iteracja
nowa trasa jest dluzsza lub rowna
[3 0 4 2 1]
18

4 iteracja
nowa trasa jest dluzsza lub rowna
[3 0 4 2 1]
18

5 iteracja
nowa trasa jest dluzsza lub rowna
[3 0 4 2 1]
18


In [289]:
#algorytm z insercją
trasa = np.arange(len(distance))
np.random.shuffle(trasa)
licza_iteracji = 5

print("Start")
print(trasa)
print(Odleglosc(distance,trasa))
for i in range(licza_iteracji):
    print("\n{} iteracja".format(i+1))
    trasa = Ruch_insercja(distance,trasa)
    print(trasa)
    print(Odleglosc(distance,trasa))

Start
[4 2 3 0 1]
16

1 iteracja
nowa trasa jest dluzsza lub rowna
[4 2 3 0 1]
16

2 iteracja
nowa trasa jest dluzsza lub rowna
[4 2 3 0 1]
16

3 iteracja
[4 3 0 1 2]
10

4 iteracja
nowa trasa jest dluzsza lub rowna
[4 3 0 1 2]
10

5 iteracja
nowa trasa jest dluzsza lub rowna
[4 3 0 1 2]
10


In [343]:
#algorytm z zamianą
trasa = np.arange(len(distance))
np.random.shuffle(trasa)
licza_iteracji = 5

print("Start")
print(trasa)
print(Odleglosc(distance,trasa))
for i in range(licza_iteracji):
    print("\n{} iteracja".format(i+1))
    trasa = Ruch_zamiana(distance,trasa)
    print(trasa)
    print(Odleglosc(distance,trasa))

Start
[3 2 1 4 0]
21

1 iteracja
[1 2 3 4 0]
18

2 iteracja
nowa trasa jest dluzsza lub rowna
[1 2 3 4 0]
18

3 iteracja
nowa trasa jest dluzsza lub rowna
[1 2 3 4 0]
18

4 iteracja
[1 2 4 3 0]
10

5 iteracja
nowa trasa jest dluzsza lub rowna
[1 2 4 3 0]
10
